<a href="https://colab.research.google.com/github/anish-mulay07/Reinforcement-Learning/blob/main/Options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 8 - Options

Please complete this tutorial to get an overview of options and an implementation of SMDP Q-Learning and Intra-Option Q-Learning.


### References:

 [Recent Advances in Hierarchical Reinforcement
Learning](https://people.cs.umass.edu/~mahadeva/papers/hrl.pdf) is a strong recommendation for topics in HRL that was covered in class. Watch Prof. Ravi's lectures on moodle or nptel for further understanding the core concepts. Contact the TAs for further resources if needed.


In [ ]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
import random
import gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML


In [ ]:
'''
The environment used here is extremely similar to the openai gym ones.
At first glance it might look slightly different.
The usual commands we use for our experiments are added to this cell to aid you
work using this environment.
'''

#Setting up the environment
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
env = CliffWalkingEnv()

env.reset()

#Current State
print(env.s)

# 4x12 grid = 48 states
print ("Number of states:", env.nS)

# Primitive Actions
action = ["up", "right", "down", "left"]
#correspond to [0,1,2,3] that's actually passed to the environment

# either go left, up, down or right
print ("Number of actions that an agent can take:", env.nA)

# Example Transitions
rnd_action = random.randint(0, 3)
print ("Action taken:", action[rnd_action])
next_state, reward, is_terminal, t_prob, _ = env.step(rnd_action)
print ("Transition probability:", t_prob)
print ("Next state:", next_state)
print ("Reward recieved:", reward)
print ("Terminal state:", is_terminal)
env.render()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


36
Number of states: 48
Number of actions that an agent can take: 4
Action taken: left
Transition probability: False
Next state: 36
Reward recieved: -1
Terminal state: False


#### Options
We custom define very simple options here. They might not be the logical options for this settings deliberately chosen to visualise the Q Table better.


In [ ]:
# We are defining two more options here
# Option 1 ["Away"] - > Away from Cliff (ie keep going up)
# Option 2 ["Close"] - > Close to Cliff (ie keep going down)

def Away(env,state):

    optdone = False
    optact = 0

    if (int(state/12) == 0):
        optdone = True

    return [optact,optdone]

def Close(env,state):

    optdone = False
    optact = 2

    if (int(state/12) == 2):
        optdone = True

    return [optact,optdone]


'''
Now the new action space will contain
Primitive Actions: ["up", "right", "down", "left"]
Options: ["Away","Close"]
Total Actions :["up", "right", "down", "left", "Away", "Close"]
Corresponding to [0,1,2,3,4,5]
'''

'\nNow the new action space will contain\nPrimitive Actions: ["up", "right", "down", "left"]\nOptions: ["Away","Close"]\nTotal Actions :["up", "right", "down", "left", "Away", "Close"]\nCorresponding to [0,1,2,3,4,5]\n'

# Task 1
Complete the code cell below


In [ ]:
#Q-Table: (States x Actions) === (env.ns(48) x total actions(6))
q_values_SMDP = np.zeros((48,6))
q_values_IOQL = np.zeros((48, 6))

#Update_Frequency Data structure? Check TODO 4
update_frequency_smdp = np.zeros((48, 6))  # Same shape as q_values_SMDP
update_frequency_IOQL = np.zeros((48, 6))

# TODO: epsilon-greedy action selection function
def egreedy_policy(q_values,state,epsilon):
  if random.uniform(0, 1) < epsilon:
        # Exploration: choose a random action
        action = np.random.choice(env.nA + 2)  # +2 to account for the options "Away" and "Close"
  else:
        # Exploitation: choose the action with the highest Q-value
        action = np.argmax(q_values[state])

  return action

# Task 2
Below is an incomplete code cell with the flow of SMDP Q-Learning. Complete the cell and train the agent using SMDP Q-Learning algorithm.
Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)

In [ ]:
#### SMDP Q-Learning

# Add parameters you might need here
gamma = 0.9
alpha = 0.9
total_rewards = 0

# Iterate over 1000 episodes
for _ in tqdm(range(1000)):
    state = env.reset()
    done = False

    # While episode is not over
    while not done:

        # Choose action
        action = egreedy_policy(q_values_SMDP, state, epsilon=0.1)

        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done, _, _ = env.step(action)
            max_next_q_value = np.max(q_values_SMDP[next_state])
            q_values_SMDP[state][action] += alpha * (reward + gamma * max_next_q_value - q_values_SMDP[state][action])

            # Update the update frequency
            update_frequency_smdp[state][action] += 1


        # Checking if action chosen is an option
        reward_bar = 0
        if action == 4: # action => Away option

            optdone = False
            while (optdone == False):

                # Think about what this function might do?
                optact,optdone = Away(env,state)
                next_state, reward, done,_,_ = env.step(optact)

                # Is this formulation right? What is this term?
                reward_bar = gamma*reward_bar + reward

                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                q_values_SMDP[state][action] += alpha * (reward_bar - q_values_SMDP[state][action])

                # Update the update frequency
                update_frequency_smdp[state][action] += 1


                state = next_state

        if action == 5: # action => Close option
          optdone = False
          while not optdone:
                # Execute the Close option
                optact, optdone = Close(env, state)
                next_state, reward, done, _, _ = env.step(optact)

                # Update reward_bar using the cumulative reward
                reward_bar = gamma * reward_bar + reward

                # Complete SMDP Q-Learning Update for the Close option
                q_values_SMDP[state][action] += alpha * (reward_bar - q_values_SMDP[state][action])

                # Update the update frequency
                update_frequency_smdp[state][action] += 1

                state = next_state



KeyboardInterrupt: 

# Task 3
Using the same options and the SMDP code, implement Intra Option Q-Learning (In the code cell below). You *might not* always have to search through options to find the options with similar policies, think about it. Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)



In [ ]:
#### Intra-Option Q-Learning

#

#

#

#

#

# Add parameters you might need here
gamma = 0.9
alpha = 0.1  # Learning rate

# Iterate over 1000 episodes
for _ in range(1000):
    state = env.reset()
    done = False

    # While episode is not over
    while not done:

        # Choose action
        action = egreedy_policy(q_values_IOQL, state, epsilon=0.1)

        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done, _ = env.step(action)
            max_next_q_value = np.max(q_values_IOQL[next_state])
            q_values_IOQL[state][action] += alpha * (reward + gamma * max_next_q_value - q_values_IOQL[state][action])

            # Update the update frequency
            update_frequency_IOQL[state][action] += 1

        # Checking if action chosen is an option
        reward_bar = 0
        if action == 4: # action => Away option

            optdone = False
            while not optdone:
                # Execute the Away option
                optact, optdone = Away(env, state)
                next_state, reward, done, _ = env.step(optact)

                # Update reward_bar using the cumulative reward
                reward_bar = gamma * reward_bar + reward

                # Complete Intra Option Q-Learning Update for the Away option
                q_values_IOQL[state][action] += alpha * (reward + gamma * np.max(q_values_IOQL[next_state]) - q_values_IOQL[state][action])

                # Update the update frequency
                update_frequency_IOQL[state][action] += 1

                state = next_state

        if action == 5: # action => Close option
            optdone = False
            while not optdone:
                # Execute the Close option
                optact, optdone = Close(env, state)
                next_state, reward, done, _ = env.step(optact)

                # Update reward_bar using the cumulative reward
                reward_bar = gamma * reward_bar + reward

                # Complete Intra Option Q-Learning Update for the Close option
                q_values_IOQL[state][action] += alpha * (reward + gamma * np.max(q_values_IOQL[next_state]) - q_values_IOQL[state][action])

                # Update the update frequency
                update_frequency_IOQL[state][action] += 1

                state = next_state


# Task 4
Compare the two Q-Tables and Update Frequencies and provide comments.

In [ ]:
# Use this cell for Task 4 Code

Use this text cell for your comments - Task 4
